In [25]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
import numpy as np
from math import pi
from urllib.request import urlopen
import matplotlib.patheffects as pe
from highlight_text import fig_text
from adjustText import adjust_text
from tabulate import tabulate
import matplotlib.style as style
import unicodedata
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib.ticker as ticker
import matplotlib.patheffects as path_effects
import matplotlib.font_manager as fm
import matplotlib.colors as mcolors
from matplotlib import cm
from highlight_text import fig_text

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

style.use('fivethirtyeight')

from PIL import Image
import urllib
import os
import math
from PIL import Image
import matplotlib.image as image
pd.options.display.max_columns = None

In [26]:
fbref_passing = 'https://fbref.com/en/comps/32/passing/Primeira-Liga-Stats'
fbref_shooting = 'https://fbref.com/en/comps/32/shooting/players/Primeira-Liga-Stats'
fbref_pass_type = 'https://fbref.com/en/comps/32/passing_types/players/Primeira-Liga-Stats'
fbref_defence = 'https://fbref.com/en/comps/32/defense/players/Primeira-Liga-Stats'
fbref_gca = 'https://fbref.com/en/comps/32/gca/players/Primeira-Liga-Stats'
fbref_poss = 'https://fbref.com/en/comps/32/possession/players/Primeira-Liga-Stats'
fbref_misc = 'https://fbref.com/en/comps/32/misc/players/Primeira-Liga-Stats'

In [27]:
keepers = ['GK']
defenders = ['DF', 'DF,MF']
wing_backs = ['FW,DF', 'DF,FW']
midfielders = ['MF', 'MF,DF']
center_forwards = ['FW']
attacking_mids = ['MF,FW', 'FW,MF']

def position_grouping(x):
    if x in keepers:
        return "GK"
    elif x in defenders:
        return "Defender"
    elif x in wing_backs:
        return "Wing-Back"
    elif x in midfielders:
        return "Central Midfielder"
    elif x in center_forwards:
        return "Center Forward"
    elif x in attacking_mids:
        return "Attacking Mid"
    else:
        return "Unidentified"


In [28]:
def create_full_stats_db(fbref_passing,fbref_shooting,fbref_pass_type,fbref_defence,fbref_gca,fbref_poss,fbref_misc):
    # Passing columns 
    pass_ = fbref_passing
    page =requests.get(pass_)
    soup = BeautifulSoup(page.content, 'html.parser')
    html_content = requests.get(pass_).text.replace('<!--', '').replace('-->', '')
    pass_df = pd.read_html(html_content)
    pass_df[-1].columns = pass_df[-1].columns.droplevel(0)
    pass_stats = pass_df[-1]
    pass_prefixes = {1: 'Total - ', 2: 'Short - ', 3: 'Medium - ', 4: 'Long - '}
    pass_column_occurrences = {'Cmp': 0, 'Att': 0, 'Cmp%': 0}
    pass_new_column_names = []
    for col_name in pass_stats.columns:
        if col_name in pass_column_occurrences:
            pass_column_occurrences[col_name] += 1
            prefix = pass_prefixes[pass_column_occurrences[col_name]]
            pass_new_column_names.append(prefix + col_name)
        else:
            pass_new_column_names.append(col_name)
    pass_stats.columns = pass_new_column_names
    pass_stats = pass_stats[pass_stats['Player'] != 'Player']

    # Shooting columns 
    shot_ = fbref_shooting
    page =requests.get(shot_)
    soup = BeautifulSoup(page.content, 'html.parser')
    html_content = requests.get(shot_).text.replace('<!--', '').replace('-->', '')
    shot_df = pd.read_html(html_content)
    shot_df[-1].columns = shot_df[-1].columns.droplevel(0) # drop top header row
    shot_stats = shot_df[-1]
    shot_stats = shot_stats[shot_stats['Player'] != 'Player']    



    # Pass Type columns 
    pass_type = fbref_pass_type
    page =requests.get(pass_type)
    soup = BeautifulSoup(page.content, 'html.parser')
    html_content = requests.get(pass_type).text.replace('<!--', '').replace('-->', '')
    pass_type_df = pd.read_html(html_content)
    pass_type_df[-1].columns = pass_type_df[-1].columns.droplevel(0) # drop top header row
    pass_type_stats = pass_type_df[-1]
    pass_type_stats = pass_type_stats[pass_type_stats['Player'] != 'Player']


    # GCA columns 
    gca_ = fbref_gca
    page =requests.get(gca_)
    soup = BeautifulSoup(page.content, 'html.parser')
    html_content = requests.get(gca_).text.replace('<!--', '').replace('-->', '')
    gca_df = pd.read_html(html_content)
    gca_df[-1].columns = gca_df[-1].columns.droplevel(0)
    gca_stats = gca_df[-1]
    gca_prefixes = {1: 'SCA - ', 2: 'GCA - '}
    gca_column_occurrences = {'PassLive': 0, 'PassDead': 0, 'TO%': 0, 'Sh': 0, 'Fld': 0, 'Def': 0}
    gca_new_column_names = []
    for col_name in gca_stats.columns:
        if col_name in gca_column_occurrences:
            gca_column_occurrences[col_name] += 1
            prefix = gca_prefixes[gca_column_occurrences[col_name]]
            gca_new_column_names.append(prefix + col_name)
        else:
            gca_new_column_names.append(col_name)
    gca_stats.columns = gca_new_column_names
    gca_stats = gca_stats[gca_stats['Player'] != 'Player']
    

    # Defense columns 
    defence_ = fbref_defence
    page =requests.get(defence_)
    soup = BeautifulSoup(page.content, 'html.parser')
    html_content = requests.get(defence_).text.replace('<!--', '').replace('-->', '')
    defence_df = pd.read_html(html_content)
    defence_df[-1].columns = defence_df[-1].columns.droplevel(0) # drop top header row
    defence_stats = defence_df[-1]
    rename_columns = {
    'Def 3rd': 'Tackles - Def 3rd',
    'Mid 3rd': 'Tackles - Mid 3rd',
    'Att 3rd': 'Tackles - Att 3rd',
    'Blocks': 'Total Blocks',
    'Sh': 'Shots Blocked',
    'Pass': 'Passes Blocked'}
    defence_stats.rename(columns = rename_columns, inplace=True)
    defence_prefixes = {1: 'Total - ', 2: 'Dribblers- '}
    defence_column_occurrences = {'Tkl': 0}
    new_column_names = []
    for col_name in defence_stats.columns:
        if col_name in defence_column_occurrences:
            defence_column_occurrences[col_name] += 1
            prefix = defence_prefixes[defence_column_occurrences[col_name]]
            new_column_names.append(prefix + col_name)
        else:
            new_column_names.append(col_name)
    defence_stats.columns = new_column_names
    defence_stats = defence_stats[defence_stats['Player'] != 'Player']


    # possession columns 
    poss_ = fbref_poss
    page =requests.get(poss_)
    soup = BeautifulSoup(page.content, 'html.parser')
    html_content = requests.get(poss_).text.replace('<!--', '').replace('-->', '')
    poss_df = pd.read_html(html_content)
    poss_df[-1].columns = poss_df[-1].columns.droplevel(0) # drop top header row
    poss_stats = poss_df[-1]
    rename_columns = {
    'TotDist': 'Carries - TotDist',
    'PrgDist': 'Carries - PrgDist',
    'PrgC': 'Carries - PrgC',
    '1/3': 'Carries - 1/3',
    'CPA': 'Carries - CPA',
    'Mis': 'Carries - Mis',
    'Dis': 'Carries - Dis',
    'Att': 'Take Ons - Attempted'  }
    poss_stats.rename(columns=rename_columns, inplace=True)
    poss_stats = poss_stats[poss_stats['Player'] != 'Player']


    # misc columns 
    misc_ = fbref_misc
    page =requests.get(misc_)
    soup = BeautifulSoup(page.content, 'html.parser')
    html_content = requests.get(misc_).text.replace('<!--', '').replace('-->', '')
    misc_df = pd.read_html(html_content)
    misc_df[-1].columns = misc_df[-1].columns.droplevel(0) # drop top header row
    misc_stats = misc_df[-1]
    misc_stats = misc_stats[misc_stats['Player'] != 'Player']

    index_df = misc_stats[['Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s']]

    data_frames = [poss_stats, misc_stats, pass_stats ,defence_stats, shot_stats, gca_stats, pass_type_stats]
    for df in data_frames:
        if df is not None:  # Checking if the DataFrame exists
            df.drop(columns=['Matches', 'Rk', 'Comp'], inplace=True, errors='ignore')
            df.dropna(axis=0, how='any', inplace=True)

            index_df = pd.merge(index_df, df, on=['Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s'], how='left')
    index_df["position_group"] = index_df.Pos.apply(lambda x: position_grouping(x))  

    index_df.fillna(0, inplace=True)

    non_numeric_cols = ['Player', 'Nation', 'Pos', 'Squad', 'Age', 'position_group']
    
    def clean_non_convertible_values(value):
        try:
            return pd.to_numeric(value)
        except (ValueError, TypeError):
            return np.nan

    index_df = index_df.reset_index()

    # Iterate through each column, converting non-numeric columns to numeric
    for col in index_df.columns:
        if col not in non_numeric_cols:
            index_df[col] = index_df[col].apply(clean_non_convertible_values)

    
    return index_df



In [29]:
def per_90fi(dataframe):
    # Replace empty strings ('') with NaN
    dataframe = dataframe.replace('', np.nan)
    
    # Fill NaN values with 0
    dataframe = dataframe.fillna(0)
    
    # Identify numeric columns excluding '90s' and columns with '90' or '%' in their names
    exclude_columns = ['index', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'position_group']
    numeric_columns = [col for col in dataframe.columns 
                        if pd.api.types.is_numeric_dtype(dataframe[col])  # Check if column is numeric
                        and col != '90s'  # Exclude '90s' column
                        and not any(exc_col in col for exc_col in exclude_columns)  # Exclude non-numeric columns
                        and ('90' not in col)  # Exclude columns with '90' in their names
                        and ('%' not in col)]  # Exclude columns with '%' in their names
    
    # Create a mask to avoid division by zero
    mask = (dataframe['90s'] != 0)
    
    # Divide each numeric column by the '90s' column row-wise
    dataframe.loc[mask, numeric_columns] = dataframe.loc[mask, numeric_columns].div(dataframe.loc[mask, '90s'], axis=0)

    return dataframe


In [30]:
stats = create_full_stats_db(fbref_passing,fbref_shooting,fbref_pass_type,fbref_defence,fbref_gca,fbref_poss,fbref_misc)

/var/folders/64/gffmh1qd2cl426g_n8z6k9kh0000gn/T/ipykernel_95156/1745367248.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pass_df = pd.read_html(html_content)
/var/folders/64/gffmh1qd2cl426g_n8z6k9kh0000gn/T/ipykernel_95156/1745367248.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shot_df = pd.read_html(html_content)
/var/folders/64/gffmh1qd2cl426g_n8z6k9kh0000gn/T/ipykernel_95156/1745367248.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pass_type_df = pd.read_html(html_content)
/var/folders/64/gffmh1qd2cl426g_n8z6k9kh0000gn/T/ipykernel_95156/1745367248.py:51: FutureWarning: Passing literal html to '

In [31]:
stats

,index,Player,Nation,Pos,Squad,Age,Born,90s,Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Live_x,Take Ons - Attempted,Succ,Succ%,Tkld,Tkld%,Carries,Carries - TotDist,Carries - PrgDist,Carries - PrgC,Carries - 1/3,Carries - CPA,Carries - Mis,Carries - Dis,Rec,PrgR,CrdY,CrdR,2CrdY,Fls,Fld,Off_x,Crs_x,Int_x,TklW_x,PKwon,PKcon,OG,Recov,Won,Lost_x,Won%,Total - Cmp,Total - Att,Total - Cmp%,TotDist,PrgDist,Short - Cmp,Short - Att,Short - Cmp%,Medium - Cmp,Medium - Att,Medium - Cmp%,Long - Cmp,Long - Att,Long - Cmp%,Ast,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,PrgP,Total - Tkl,TklW_y,Tackles - Def 3rd,Tackles - Mid 3rd,Tackles - Att 3rd,Dribblers- Tkl,Att_x,Tkl%,Lost_y,Total Blocks,Shots Blocked,Passes Blocked,Int_y,Tkl+Int,Clr,Err,Gls,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK_x,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,SCA,SCA90,SCA - PassLive,SCA - PassDead,TO,SCA - Sh,SCA - Fld,SCA - Def,GCA,GCA90,GCA - PassLive,GCA - PassDead,TO,GCA - Sh,GCA - Fld,GCA - Def,Att_y,Live_y,Dead,FK_y,TB,Sw,Crs_y,TI,CK,In,Out,Str,Cmp,Off_y,Blocks,position_group
0,0,Rodrigo Abascal,uy URU,DF,Boavista,30,1994,28.9,1607,222,849,589,179,23,1607,12,7,58.3,5,41.7,744,3182,1552,10,6,0,13,4,813,24,8,1,0,35,6,5,32,40,33,0,1,1,99,39,30,56.5,904,1286,70.3,21266,11349,216,246,87.8,476,578,82.4,207,440,47.0,0,1.7,1.5,-1.7,17,84,15,6,66,55,33,39,15,1,30,57,52.6,27,40,26,14,40,95,144,3,0,19,5,26.3,0.66,0.17,0.00,0.00,24.0,2,0,0,0.8,0.8,0.04,-0.8,-0.8,48,1.66,29,16,1,2,0,0,1,0.03,1,0,0,0,0,0,1286,1072,209,93,2,16,32,104,1,1,0,0,904,5,10,Defender
1,1,Nelson Abbey,eng ENG,DF,Rio Ave,20,2003,7.1,460,52,223,196,44,3,460,8,3,37.5,4,50.0,229,1220,574,7,7,1,8,5,253,7,1,0,0,5,5,0,7,5,7,0,0,0,30,6,6,50.0,279,366,76.2,4917,1600,115,130,88.5,146,169,86.4,15,49,30.6,0,0.1,0.1,-0.1,2,9,3,2,14,12,7,7,2,3,7,8,87.5,1,10,4,6,5,17,46,1,0,0,0,0.0,0.00,0.00,0.00,0.00,0.0,0,0,0,0.0,0.0,0.00,0.0,0.0,4,0.57,3,1,0,0,0,0,0,0.00,0,0,0,0,0,0,366,304,59,17,0,2,7,39,0,0,0,0,279,3,12,Defender
2,2,Bilal Abdelrahman Zeeni,eg EGY,MF,Estrela,21,2003,0.1,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,50.0,0,0,0.0,0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,1,1,100.0,11.25,11.25,0.00,0.00,25.2,0,0,0,0.0,0.0,0.02,0.0,0.0,3,30.00,2,0,0,0,0,1,0,0.00,0,0,0,0,0,0,3,3,0,0,0,0,0,0,0,0,0,0,2,0,0,Central Midfielder
3,3,Giorgi Aburjania,ge GEO,MF,AVS Futebol,29,1995,3.4,167,7,30,109,30,3,167,2,1,50.0,0,0.0,71,288,181,3,3,0,4,1,102,4,0,0,0,3,5,0,2,3,6,0,0,0,13,4,2,66.7,115,138,83.3,1907,572,58,63,92.1,49,58,84.5,8,13,61.5,0,0.6,0.2,-0.6,9,17,1,0,22,11,6,3,6,2,7,13,53.8,6,3,0,3,3,14,5,0,0,1,1,100.0,0.29,0.29,0.00,0.00,10.8,0,0,0,0.3,0.3,0.32,-0.3,-0.3,13,3.82,13,0,0,0,0,0,0,0.00,0,0,0,0,0,0,138,129,8,7,0,2,2,0,1,1,0,0,115,1,2,Central Midfielder
4,4,Adriano,br BRA,MF,Santa Clara,24,1999,29.2,1418,71,370,868,202,14,1418,48,23,47.9,24,50.0,758,3758,1738,26,23,1,21,18,945,28,8,1,0,41,20,2,14,17,18,0,1,0,152,17,24,41.5,1004,1193,84.2,17150,4413,445,495,89.9,461,525,87.8,73,123,59.3,2,2.7,1.5,-0.7,19,106,10,2,99,38,18,12,23,3,20,36,55.6,16,25,7,18,17,55,52,1,0,13,4,30.8,0.44,0.14,0.00,0.00,18.6,0,0,0,1.0,1.0,0.08,-1.0,-1.0,54,1.85,46,0,1,3,1,3,5,0.17,4,0,0,1,0,0,1193,1141,47,43,3,5,14,4,0,0,0,0,1004,5,12,Central Midfielder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,580,Rodrigo Zalazar,uy URU,"MF,FW",Braga,24,1999,15.4,957,14,65,478,433,66,956,73,34,46.6,31,42.5,552,3548,1749,65,36,12,46,30,692,116,4,0,0,22,39,0,82,5,14,0,0,0,93,39,32,54.9,540,720,75.0,9470,2302,264,3

In [32]:
# position = "Forwards"  

In [33]:
def key_stats_db(df):
    non_numeric_cols = ['Player', 'Nation', 'Pos', 'Squad', 'Age', 'position_group']
    core_stats = ['90s','Total - Cmp%','KP', 'TB','Sw','PPA', 'PrgP','Tkl%','Blocks', 'Tkl+Int','Clr', 'Carries - PrgDist','SCA90','GCA90','CrsPA','xA', 'Rec','PrgR','xG', 'Sh','SoT']
    df.dropna(axis=0, how='any', inplace=True)
    # key_stats_df = df[df['position_group'] == position]
    key_stats_df = df
    key_stats_df = key_stats_df[non_numeric_cols + core_stats]
    key_stats_df = key_stats_df[key_stats_df['90s'] > 5]
    # key_stats_df = per_90fi(key_stats_df)
    return key_stats_df


key_stats_df = key_stats_db(stats)

In [34]:
key_stats_df

,Player,Nation,Pos,Squad,Age,position_group,90s,Total - Cmp%,KP,TB,Sw,PPA,PrgP,Tkl%,Blocks,Tkl+Int,Clr,Carries - PrgDist,SCA90,GCA90,CrsPA,xA,Rec,PrgR,xG,Sh,SoT
0,Rodrigo Abascal,uy URU,DF,Boavista,30,Defender,28.9,70.3,17,2,16,15,66,52.6,10,95,144,1552,1.66,0.03,6,1.5,813,24,0.8,19,5
1,Nelson Abbey,eng ENG,DF,Rio Ave,20,Defender,7.1,76.2,2,0,2,3,14,87.5,12,17,46,574,0.57,0.00,2,0.1,253,7,0.0,0,0
4,Adriano,br BRA,MF,Santa Clara,24,Central Midfielder,29.2,84.2,19,3,5,10,99,55.6,12,55,52,1738,1.85,0.17,2,1.5,945,28,1.0,13,4
5,Lucas Áfrico,br BRA,DF,Farense,29,Defender,19.5,79.0,3,0,14,1,31,71.4,14,39,99,1225,0.67,0.05,0,0.1,545,3,1.3,16,3
6,Salvador Agra,pt POR,"MF,DF",Boavista,32,Central Midfielder,30.3,67.9,53,2,6,28,73,42.4,28,46,19,1694,2.83,0.10,17,3.8,730,147,1.4,33,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,Zaydou Youssouf,km COM,MF,Famalicão,25,Central Midfielder,14.8,81.2,11,5,22,8,50,56.7,5,51,12,816,1.55,0.07,3,1.0,525,38,2.2,18,7
580,Rodrigo Zalazar,uy URU,"MF,FW",Braga,24,Attacking Mid,15.4,75.0,29,5,12,26,76,37.0,21,38,12,1749,3.37,0.45,10,2.6,692,116,3.6,42,6
581,Vinícius Zanocelo,br BRA,MF,Estoril,23,Central Midfielder,23.1,83.9,9,2,5,6,106,55.6,16,74,56,1282,1.56,0.30,1,0.7,879,34,2.2,29,8
582,Ivan Zlobin,ru RUS,GK,Famalicão,27,GK,15.0,71.3,0,0,0,0,0,0.0,1,0,0,0,0.00,0.00,0,0.0,0,0,0.0,0,0


In [35]:
def create_metrics_scores(key_stats_df):
    # Define the key_stats grouped by the metrics
    core_stats = ['90s','Total - Cmp%','KP', 'TB','Sw','PPA', 'PrgP','Tkl%','Blocks', 'Tkl+Int','Clr', 'Carries - PrgDist','SCA90','GCA90','CrsPA','xA', 'Rec','PrgR','xG', 'Sh','SoT']
    passing_metrics = ['Total - Cmp%', 'KP', 'TB', 'Sw', 'PPA', 'PrgP']
    defending_metrics = ['Tkl%', 'Blocks', 'Tkl+Int', 'Clr']
    creation_metrics = ['Carries - PrgDist', 'SCA90', 'GCA90', 'CrsPA', 'xA', 'Rec', 'PrgR']
    shooting_metrics = ['xG', 'Sh', 'SoT']

    # Create a MinMaxScaler instance
    scaler = MinMaxScaler()

    # Normalize the metrics
    stats_normalized = key_stats_df.copy()  # Create a copy of the DataFrame
    stats_normalized[core_stats] = scaler.fit_transform(stats_normalized[core_stats])

    # Calculate scores for each metric grouping and scale to 0-10
    stats_normalized['Passing_Score'] = stats_normalized[passing_metrics].mean(axis=1) * 10
    stats_normalized['Defending_Score'] = stats_normalized[defending_metrics].mean(axis=1) * 10
    stats_normalized['Creation_Score'] = stats_normalized[creation_metrics].mean(axis=1) * 10
    stats_normalized['Shooting_Score'] = stats_normalized[shooting_metrics].mean(axis=1) * 10

    # Add a small offset to ensure unique scores
    stats_normalized['Passing_Score'] += stats_normalized.index * 0.001
    stats_normalized['Defending_Score'] += stats_normalized.index * 0.001
    stats_normalized['Creation_Score'] += stats_normalized.index * 0.001
    stats_normalized['Shooting_Score'] += stats_normalized.index * 0.001

    # Clip scores to ensure they are within the 0-10 range
    stats_normalized[['Passing_Score', 'Defending_Score', 'Creation_Score', 'Shooting_Score']] = stats_normalized[['Passing_Score', 'Defending_Score', 'Creation_Score', 'Shooting_Score']].clip(lower=0, upper=10)
    return stats_normalized

def adjust_player_rating_range(dataframe):
    # Get the 'total player rating' column
    player_ratings = dataframe[['Passing_Score', 'Defending_Score', 'Creation_Score', 'Shooting_Score']]
    
    # Define the desired range for the ratings
    min_rating = 4.5
    max_rating = 9.5
    
    # Normalize the ratings to be within the desired range (5 to 9.5) for each column
    for col in player_ratings.columns:
        normalized_ratings = min_rating + (max_rating - min_rating) * ((player_ratings[col] - player_ratings[col].min()) / (player_ratings[col].max() - player_ratings[col].min()))
        dataframe[col] = normalized_ratings
    
    return dataframe


pitch_iq_scoring = create_metrics_scores(key_stats_df)
pitch_iq_scoring = adjust_player_rating_range(pitch_iq_scoring)
pitch_iq_scoring = pitch_iq_scoring[['Player','Passing_Score', 'Defending_Score', 'Creation_Score', 'Shooting_Score']]
pitch_iq_scores = pd.merge(key_stats_df, pitch_iq_scoring, on='Player', how='left')


In [36]:
liga_nos_fwds = pitch_iq_scores

In [37]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=97, limit=1):
    """
    Perform a fuzzy merge between two DataFrames based on string similarity.
    
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, sorted high to low
    :return: dataframe with both keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

def remove_accents(input_str):
    """
    Remove accents from a string, replace special characters, and convert it to ASCII.
    
    :param input_str: input string with accents and special characters
    :return: string without accents and with special characters replaced
    """
    # Replace ø with o and ð with d
    input_str = input_str.replace('ø', 'o').replace('ð', 'd').replace('Ø', 'O').replace('Ð', 'D').replace('ı', 'i')
    
    # Normalize the string to decompose accented characters
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    
    # Encode to ASCII, ignoring non-ASCII characters
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    
    # Convert bytes to string and remove the "b''" wrapper
    only_ascii = str(only_ascii)
    only_ascii = only_ascii[2:-1]
    
    # Replace hyphens with spaces
    only_ascii = only_ascii.replace('-', ' ')
    
    return only_ascii

def years_converter(variable_value):
    """
    Convert age values in the format 'YYYY-DDDD' to a float representing years.
    
    :param variable_value: age value in 'YYYY-DDDD' format
    :return: age as a float
    """
    if len(variable_value) > 3:
        years = variable_value[:-4]
        days = variable_value[3:]
        years_value = pd.to_numeric(years)
        days_value = pd.to_numeric(days)
        day_conv = days_value / 365
        final_val = years_value + day_conv
    else:
        final_val = pd.to_numeric(variable_value)

    return final_val

In [38]:
player_phonebook = pd.read_csv('/Users/stephenahiabah/Desktop/Code/cannoniq/CSVs/24-25_w_liga_nos_player_phonebook.csv')

In [39]:
# Step 1: Apply remove_accents to the Player column in forwards_db
liga_nos_fwds['player_name_match'] = liga_nos_fwds['Player'].apply(lambda x: remove_accents(x))

# Step 2: Rename the Player column in player_phonebook
player_phonebook = player_phonebook.rename(columns={'Player': 'player_name_match'})

# Step 3: Check for duplicates in player_phonebook
duplicates = player_phonebook[player_phonebook.duplicated('player_name_match', keep=False)]
print("Duplicates in player_phonebook:")
# print(duplicates)

# Step 4: Remove duplicates in player_phonebook (keep first occurrence)
player_phonebook = player_phonebook.drop_duplicates(subset='player_name_match', keep='first')

# Step 5: Join the DataFrames
liga_nos_fwds_extended = pd.merge(
    liga_nos_fwds,  # Left DataFrame
    player_phonebook[['player_name_match', 'scouting_url', 'match_logs']],  # Right DataFrame
    on='player_name_match',  # Join key
    how='left'  # Keep all rows from forwards_db
)

# Step 6: Verify the result
duplicates_merged = liga_nos_fwds_extended[liga_nos_fwds_extended.duplicated('Player', keep=False)]
print("Duplicates in merged DataFrame:")
# print(duplicates_merged)

# Display the merged DataFrame
liga_nos_fwds_extended

Duplicates in player_phonebook:
Duplicates in merged DataFrame:


,Player,Nation,Pos,Squad,Age,position_group,90s,Total - Cmp%,KP,TB,Sw,PPA,PrgP,Tkl%,Blocks,Tkl+Int,Clr,Carries - PrgDist,SCA90,GCA90,CrsPA,xA,Rec,PrgR,xG,Sh,SoT,Passing_Score,Defending_Score,Creation_Score,Shooting_Score,player_name_match,scouting_url,match_logs
0,Rodrigo Abascal,uy URU,DF,Boavista,30,Defender,28.9,70.3,17,2,16,15,66,52.6,10,95,144,1552,1.66,0.03,6,1.5,813,24,0.8,19,5,6.027140,8.043843,5.985427,4.918438,Rodrigo Abascal,https://fbref.com/en/players/088ae0d1/scout/36...,https://fbref.com/en/players/088ae0d1/matchlog...
1,Nelson Abbey,eng ENG,DF,Rio Ave,20,Defender,7.1,76.2,2,0,2,3,14,87.5,12,17,46,574,0.57,0.00,2,0.1,253,7,0.0,0,0,5.187530,7.022547,4.924744,4.500000,Nelson Abbey,https://fbref.com/en/players/33f40a36/scout/36...,https://fbref.com/en/players/33f40a36/matchlog...
2,Adriano,br BRA,MF,Santa Clara,24,Central Midfielder,29.2,84.2,19,3,5,10,99,55.6,12,55,52,1738,1.85,0.17,2,1.5,945,28,1.0,13,4,6.060539,6.953585,6.149939,4.827259,Adriano,https://fbref.com/en/players/81053aea/scout/36...,https://fbref.com/en/players/81053aea/matchlog...
3,Lucas Áfrico,br BRA,DF,Farense,29,Defender,19.5,79.0,3,0,14,1,31,71.4,14,39,99,1225,0.67,0.05,0,0.1,545,3,1.3,16,3,5.502324,7.501526,5.213594,4.858113,Lucas Africo,https://fbref.com/en/players/8fd083bb/scout/36...,https://fbref.com/en/players/8fd083bb/matchlog...
4,Salvador Agra,pt POR,"MF,DF",Boavista,32,Central Midfielder,30.3,67.9,53,2,6,28,73,42.4,28,46,19,1694,2.83,0.10,17,3.8,730,147,1.4,33,7,6.474772,6.926965,7.227308,5.188165,Salvador Agra,https://fbref.com/en/players/a130741a/scout/36...,https://fbref.com/en/players/a130741a/matchlog...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,Zaydou Youssouf,km COM,MF,Famalicão,25,Central Midfielder,14.8,81.2,11,5,22,8,50,56.7,5,51,12,816,1.55,0.07,3,1.0,525,38,2.2,18,7,6.516000,6.755324,5.975246,5.320618,Zaydou Youssouf,https://fbref.com/en/players/e2f9bb5f/scout/36...,https://fbref.com/en/players/e2f9bb5f/matchlog...
395,Rodrigo Zalazar,uy URU,"MF,FW",Braga,24,Attacking Mid,15.4,75.0,29,5,12,26,76,37.0,21,38,12,1749,3.37,0.45,10,2.6,692,116,3.6,42,6,6.816378,6.834059,7.569836,5.687120,Rodrigo Zalazar,https://fbref.com/en/players/fc3697dd/scout/36...,https://fbref.com/en/players/fc3697dd/matchlog...
396,Vinícius Zanocelo,br BRA,MF,Estoril,23,Central Midfielder,23.1,83.9,9,2,5,6,106,55.6,16,74,56,1282,1.56,0.30,1,0.7,879,34,2.2,29,8,6.192617,7.766106,6.382635,5.491744,Vinicius Zanocelo,https://fbref.com/en/players/27457e83/scout/36...,https://fbref.com/en/players/27457e83/matchlog...
397,Ivan Zlobin,ru RUS,GK,Famalicão,27,GK,15.0,71.3,0,0,0,0,0,0.0,1,0,0,0,0.00,0.00,0,0.0,0,0,0.0,0,0,5.329239,4.905095,4.885209,4.790529,Ivan Zlobin,https://fbref.com/en/players/d8fe652c/scout/36...,https://fbref.com/en/players/d8fe652c/matchlog...


In [40]:
stats = per_90fi(stats)

liga_nos_fwds_extended_cut = liga_nos_fwds_extended[['Player', 'Nation', 'Pos', 'Squad', 'position_group','Passing_Score', 'Defending_Score', 'Creation_Score', 'Shooting_Score','player_name_match', 'scouting_url','match_logs']]

liga_nos_fwds_extended = pd.merge(liga_nos_fwds_extended_cut, stats, on=['Player', 'Nation', 'Pos', 'Squad'], how='inner')

columns_to_drop = [col for col in liga_nos_fwds_extended.columns if col.endswith('_y')]
liga_nos_fwds_extended = liga_nos_fwds_extended.drop(columns=columns_to_drop)
liga_nos_fwds_extended.columns = [col.replace('_x', '') for col in liga_nos_fwds_extended.columns]
liga_nos_fwds_extended = liga_nos_fwds_extended.drop(columns=['TO', 'index'])

/var/folders/64/gffmh1qd2cl426g_n8z6k9kh0000gn/T/ipykernel_95156/566884459.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 55.60553633  64.78873239   0.          49.11764706  48.56164384
  47.33333333  40.79207921  55.4248366   31.8         39.84615385
  43.2735426   65.67567568  52.22222222  44.77777778  57.21649485
  55.97744361  48.31578947   0.           0.          43.97849462
  45.71428571  41.03806228   0.          27.80487805  62.58883249
  70.73929961  49.65986395  25.78947368  56.66666667  77.38095238
   0.          54.57142857  87.9558011   45.77777778  58.04597701
   0.           0.          56.13043478  31.30434783  75.66878981
   0.           0.          42.79069767  35.77586207  59.24528302
  76.57894737  35.           0.          47.          53.40740741
  49.16666667   0.          50.           0.          34.08450704
  42.33333333  36.2745098   52.09302326  80.5         58.554216

In [41]:
liga_nos_fwds_extended

,Player,Nation,Pos,Squad,position_group,Passing_Score,Defending_Score,Creation_Score,Shooting_Score,player_name_match,scouting_url,match_logs,Age,Born,90s,Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Live,Take Ons - Attempted,Succ,Succ%,Tkld,Tkld%,Carries,Carries - TotDist,Carries - PrgDist,Carries - PrgC,Carries - 1/3,Carries - CPA,Carries - Mis,Carries - Dis,Rec,PrgR,CrdY,CrdR,2CrdY,Fls,Fld,Off,Crs,Int,TklW,PKwon,PKcon,OG,Recov,Won,Lost,Won%,Total - Cmp,Total - Att,Total - Cmp%,TotDist,PrgDist,Short - Cmp,Short - Att,Short - Cmp%,Medium - Cmp,Medium - Att,Medium - Cmp%,Long - Cmp,Long - Att,Long - Cmp%,Ast,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,PrgP,Total - Tkl,Tackles - Def 3rd,Tackles - Mid 3rd,Tackles - Att 3rd,Dribblers- Tkl,Att,Tkl%,Total Blocks,Shots Blocked,Passes Blocked,Tkl+Int,Clr,Err,Gls,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,SCA,SCA90,SCA - PassLive,SCA - PassDead,SCA - Sh,SCA - Fld,SCA - Def,GCA,GCA90,GCA - PassLive,GCA - PassDead,GCA - Sh,GCA - Fld,GCA - Def,Dead,TB,Sw,TI,CK,In,Out,Str,Cmp,Blocks
0,Rodrigo Abascal,uy URU,DF,Boavista,Defender,6.027140,8.043843,5.985427,4.918438,Rodrigo Abascal,https://fbref.com/en/players/088ae0d1/scout/36...,https://fbref.com/en/players/088ae0d1/matchlog...,30,1994,28.9,55.605536,7.681661,29.377163,20.380623,6.193772,0.795848,55.605536,0.415225,0.242215,58.3,0.173010,41.7,25.743945,110.103806,53.702422,0.346021,0.207612,0.000000,0.449827,0.138408,28.131488,0.830450,0.276817,0.034602,0.0,1.211073,0.207612,0.173010,1.107266,1.384083,1.141869,0.0,0.034602,0.034602,3.425606,1.349481,1.038062,56.5,31.280277,44.498270,70.3,735.847751,392.698962,7.474048,8.512111,87.8,16.470588,20.000000,82.4,7.162630,15.224913,47.0,0.000000,0.058824,0.051903,-0.058824,0.588235,2.906574,0.519031,0.207612,2.283737,1.903114,1.349481,0.519031,0.034602,1.038062,1.972318,52.6,1.384083,0.899654,0.484429,3.287197,4.982699,0.103806,0.000000,0.657439,0.173010,26.3,0.66,0.17,0.000000,0.000000,0.830450,0.069204,0.000000,0.000000,0.027682,0.027682,0.001384,-0.027682,-0.027682,1.660900,1.66,1.003460,0.553633,0.069204,0.000000,0.000000,0.034602,0.03,0.034602,0.000000,0.000000,0.0,0.0,7.231834,0.069204,0.553633,3.598616,0.034602,0.034602,0.000000,0.0,31.280277,0.346021
1,Nelson Abbey,eng ENG,DF,Rio Ave,Defender,5.187530,7.022547,4.924744,4.500000,Nelson Abbey,https://fbref.com/en/players/33f40a36/scout/36...,https://fbref.com/en/players/33f40a36/matchlog...,20,2003,7.1,64.788732,7.323944,31.408451,27.605634,6.197183,0.422535,64.788732,1.126761,0.422535,37.5,0.563380,50.0,32.253521,171.830986,80.845070,0.985915,0.985915,0.140845,1.126761,0.704225,35.633803,0.985915,0.140845,0.000000,0.0,0.704225,0.704225,0.000000,0.985915,0.704225,0.985915,0.0,0.000000,0.000000,4.225352,0.845070,0.845070,50.0,39.295775,51.549296,76.2,692.535211,225.352113,16.197183,18.309859,88.5,20.563380,23.802817,86.4,2.112676,6.901408,30.6,0.000000,0.014085,0.014085,-0.014085,0.281690,1.267606,0.422535,0.281690,1.971831,1.690141,0.985915,0.281690,0.422535,0.985915,1.126761,87.5,1.408451,0.563380,0.845070,2.394366,6.478873,0.140845,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.563380,0.57,0.422535,0.140845,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,8.309859,0.000000,0.281690,5.492958,0.000000,0.000000,0.000000,0.0,39.295775,1.690141
2,Adriano,br BRA,MF,Santa Clara,Central Midfielder,6.060539,6.953585,6.149939,4.827259,Adriano,https://fbref.com/en/players/81053aea/scout/36...,https://fbref.com/en/players/81053aea/matchlog...,24,1999,29.2,48.561644,2.431507,12.671233,29.726027,6.917808,0.479452,48.561644,1.643836,0.787671,47.9,0.821918,50.0,25.958904,128.698630,59.520548,0.890411,0.787671,0.034247,0.719178,0.616438,32.363014,0.958904,0.273973,0.034247,0.0,1.404110,0.684932,0.068493,0.479452,0.582192,0.616438,0.0,0.034247,0.000000,5.205479,0.582192,0.821918,41.5,34.383562,40.856164,84.2,

In [42]:
# additional_team_ids = {
#     9772: 'Benfica',
#     9773: 'Porto',
#     9768: 'Sporting CP'
# }

# teamIDs = pd.read_csv('/Users/stephenahiabah/Desktop/Code/cannoniq/CSVs/Top6_leagues_fotmob_ids.csv')

In [43]:
# # Convert the additional_team_ids dictionary to a DataFrame
# additional_team_ids_df = pd.DataFrame(list(additional_team_ids.items()), columns=['team_id', 'team'])

# # Append the new rows to the existing teamIDs DataFrame
# teamIDs = pd.concat([teamIDs, additional_team_ids_df], ignore_index=True)
# teamIDs.to_csv('/Users/stephenahiabah/Desktop/Code/cannoniq/CSVs/Top6_leagues_fotmob_ids.csv', index=False)

In [44]:
Cannoniq_DB = pd.read_csv('/Users/stephenahiabah/Desktop/Code/cannoniq/CSVs/Cannoniq_DB.csv')

Cannoniq_DB = pd.concat([Cannoniq_DB, liga_nos_fwds_extended], ignore_index=True)
Cannoniq_DB

,Player,Nation,Pos,Squad,position_group,Passing_Score,Defending_Score,Creation_Score,Shooting_Score,player_name_match,scouting_url,match_logs,index,Age,Born,90s,Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Live,Take Ons - Attempted,Succ,Succ%,Tkld,Tkld%,Carries,Carries - TotDist,Carries - PrgDist,Carries - PrgC,Carries - 1/3,Carries - CPA,Carries - Mis,Carries - Dis,Rec,PrgR,CrdY,CrdR,2CrdY,Fls,Fld,Off,Crs,Int,TklW,PKwon,PKcon,OG,Recov,Won,Lost,Won%,Total - Cmp,Total - Att,Total - Cmp%,TotDist,PrgDist,Short - Cmp,Short - Att,Short - Cmp%,Medium - Cmp,Medium - Att,Medium - Cmp%,Long - Cmp,Long - Att,Long - Cmp%,Ast,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,PrgP,Total - Tkl,Tackles - Def 3rd,Tackles - Mid 3rd,Tackles - Att 3rd,Dribblers- Tkl,Att,Tkl%,Total Blocks,Shots Blocked,Passes Blocked,Tkl+Int,Clr,Err,Gls,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,SCA,SCA90,SCA - PassLive,SCA - PassDead,SCA - Sh,SCA - Fld,SCA - Def,GCA,GCA90,GCA - PassLive,GCA - PassDead,GCA - Sh,GCA - Fld,GCA - Def,Dead,TB,Sw,TI,CK,In,Out,Str,Cmp,Blocks
0,Max Aarons,eng ENG,DF,Bournemouth,Defender,5.644444,6.252431,5.102952,4.500000,Max Aarons,https://fbref.com/en/players/774cf58b/scout/36...,https://fbref.com/en/players/774cf58b/matchlog...,0.0,24,2000,1.0,73.000000,2.000000,19.000000,40.000000,15.000000,0.000000,73.000000,2.000000,0.000000,0.0,1.000000,50.0,41.000000,152.000000,68.000000,1.000000,0.000000,0.000000,1.000000,0.000000,40.000000,3.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,50.000000,63.000000,79.4,887.000000,361.000000,21.000000,25.000000,84.0,24.000000,29.000000,82.8,4.000000,7.000000,57.1,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000,8.000000,2.000000,1.000000,1.000000,0.000000,1.000000,1.000000,100.0,3.000000,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.09,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00000,0.0,0.0,12.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.0,50.000000,1.000000
1,Rodrigo Abajas,es ESP,DF,Valencia,Defender,5.280621,6.487013,4.972562,4.501269,Rodrigo Abajas,https://fbref.com/en/players/69c99fce/scout/36...,https://fbref.com/en/players/69c99fce/matchlog...,2.0,21,2003,0.7,51.428571,1.428571,11.428571,27.142857,12.857143,1.428571,51.428571,1.428571,1.428571,100.0,0.000000,0.0,18.571429,144.285714,95.714286,4.285714,2.857143,1.428571,0.000000,2.857143,22.857143,4.285714,1.428571,0.0,0.0,2.857143,0.000000,1.428571,1.428571,1.428571,2.857143,0.0,0.0,0.0,2.857143,0.000000,1.428571,0.0,24.285714,41.428571,58.6,382.857143,157.142857,15.714286,20.000000,78.6,5.714286,10.000000,57.1,2.857143,10.000000,28.6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.857143,4.285714,2.857143,1.428571,0.000000,4.285714,4.285714,100.0,1.428571,0.000000,1.428571,5.714286,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00000,0.0,0.0,11.428571,0.000000,0.000000,11.428571,0.000000,0.000000,0.000000,0.0,24.285714,2.857143
2,James Abankwah,ie IRL,"DF,MF",Udinese,Defender,5.550083,5.869027,4.517253,4.501903,James Abankwah,https://fbref.com/en/players/4b7a9fcc/scout/36...,https://fbref.com/en/players/4b7a9fcc/matchlog...,3.0,20,2004,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,4.000000,3.000000,0.000000,0.000000,1.000000,2.000000,0.0,0.0,0.0,7.000000,2.000000,2.000000,50.0,36.000000,46.000000,78.3,614.000000,206.000000,13.000000,15.000000,86.7,18.

In [45]:
Cannoniq_DB = Cannoniq_DB.drop_duplicates(subset=['Player'], keep='first')
Cannoniq_DB

,Player,Nation,Pos,Squad,position_group,Passing_Score,Defending_Score,Creation_Score,Shooting_Score,player_name_match,scouting_url,match_logs,index,Age,Born,90s,Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Live,Take Ons - Attempted,Succ,Succ%,Tkld,Tkld%,Carries,Carries - TotDist,Carries - PrgDist,Carries - PrgC,Carries - 1/3,Carries - CPA,Carries - Mis,Carries - Dis,Rec,PrgR,CrdY,CrdR,2CrdY,Fls,Fld,Off,Crs,Int,TklW,PKwon,PKcon,OG,Recov,Won,Lost,Won%,Total - Cmp,Total - Att,Total - Cmp%,TotDist,PrgDist,Short - Cmp,Short - Att,Short - Cmp%,Medium - Cmp,Medium - Att,Medium - Cmp%,Long - Cmp,Long - Att,Long - Cmp%,Ast,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,PrgP,Total - Tkl,Tackles - Def 3rd,Tackles - Mid 3rd,Tackles - Att 3rd,Dribblers- Tkl,Att,Tkl%,Total Blocks,Shots Blocked,Passes Blocked,Tkl+Int,Clr,Err,Gls,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,SCA,SCA90,SCA - PassLive,SCA - PassDead,SCA - Sh,SCA - Fld,SCA - Def,GCA,GCA90,GCA - PassLive,GCA - PassDead,GCA - Sh,GCA - Fld,GCA - Def,Dead,TB,Sw,TI,CK,In,Out,Str,Cmp,Blocks
0,Max Aarons,eng ENG,DF,Bournemouth,Defender,5.644444,6.252431,5.102952,4.500000,Max Aarons,https://fbref.com/en/players/774cf58b/scout/36...,https://fbref.com/en/players/774cf58b/matchlog...,0.0,24,2000,1.0,73.000000,2.000000,19.000000,40.000000,15.000000,0.000000,73.000000,2.000000,0.000000,0.0,1.000000,50.0,41.000000,152.000000,68.000000,1.000000,0.000000,0.000000,1.000000,0.000000,40.000000,3.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,50.000000,63.000000,79.4,887.000000,361.000000,21.000000,25.000000,84.0,24.000000,29.000000,82.8,4.000000,7.000000,57.1,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000,8.000000,2.000000,1.000000,1.000000,0.000000,1.000000,1.000000,100.0,3.000000,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.09,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00000,0.0,0.0,12.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.0,50.000000,1.000000
1,Rodrigo Abajas,es ESP,DF,Valencia,Defender,5.280621,6.487013,4.972562,4.501269,Rodrigo Abajas,https://fbref.com/en/players/69c99fce/scout/36...,https://fbref.com/en/players/69c99fce/matchlog...,2.0,21,2003,0.7,51.428571,1.428571,11.428571,27.142857,12.857143,1.428571,51.428571,1.428571,1.428571,100.0,0.000000,0.0,18.571429,144.285714,95.714286,4.285714,2.857143,1.428571,0.000000,2.857143,22.857143,4.285714,1.428571,0.0,0.0,2.857143,0.000000,1.428571,1.428571,1.428571,2.857143,0.0,0.0,0.0,2.857143,0.000000,1.428571,0.0,24.285714,41.428571,58.6,382.857143,157.142857,15.714286,20.000000,78.6,5.714286,10.000000,57.1,2.857143,10.000000,28.6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.857143,4.285714,2.857143,1.428571,0.000000,4.285714,4.285714,100.0,1.428571,0.000000,1.428571,5.714286,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00000,0.0,0.0,11.428571,0.000000,0.000000,11.428571,0.000000,0.000000,0.000000,0.0,24.285714,2.857143
2,James Abankwah,ie IRL,"DF,MF",Udinese,Defender,5.550083,5.869027,4.517253,4.501903,James Abankwah,https://fbref.com/en/players/4b7a9fcc/scout/36...,https://fbref.com/en/players/4b7a9fcc/matchlog...,3.0,20,2004,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,4.000000,3.000000,0.000000,0.000000,1.000000,2.000000,0.0,0.0,0.0,7.000000,2.000000,2.000000,50.0,36.000000,46.000000,78.3,614.000000,206.000000,13.000000,15.000000,86.7,18.

In [46]:
Cannoniq_DB.to_csv('/Users/stephenahiabah/Desktop/Code/cannoniq/CSVs/Cannoniq_DB_w_ln.csv', index=False)